In [11]:
import pandas as pd
import numpy as np
import seaborn as sns

In [58]:
#Data prep
shot_df = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df = shot_df.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df = shot_df[shot_df['SEASON'] != "GAME_ID"]

#convert columns to numbers
shot_df[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]] = shot_df[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]].apply(pd.to_numeric)

#convert missed/made to 0/1
shot_df['SHOT_RESULT'] = shot_df['SHOT_RESULT'].replace({'missed': 0})
shot_df['SHOT_RESULT'] = shot_df['SHOT_RESULT'].replace({'made': 1})

#classify shot distances
shot_df.loc[shot_df['PT_VALUE'] == 3, 'SHOT_TYPE'] = "Three Pointer"
shot_df.loc[(shot_df['SHOT_DIST'] >= 15) & (shot_df['PT_VALUE'] == 2), 'SHOT_TYPE'] = "Long Two-Pointer"
shot_df.loc[(shot_df['SHOT_DIST'] >= 5) & (shot_df['SHOT_DIST'] < 15), 'SHOT_TYPE'] = "Midrange"
shot_df.loc[shot_df['SHOT_DIST']< 5, 'SHOT_TYPE'] = "Layup"

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
#drop everything but shot distance
shot_df = shot_df[['SHOT_RESULT', "SHOT_DIST", "DRIBBLES", "CLOSE_DEF_DIST", "TOUCH_TIME"]]


In [67]:
from sklearn.model_selection import train_test_split


In [68]:
X_train, X_test, y_train, y_test = train_test_split(shot_df.drop('SHOT_RESULT',axis=1), 
                                                    shot_df['SHOT_RESULT'], test_size=0.3, 
                                                  random_state=101)

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
lm = LinearRegression()
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [71]:
print(lm.intercept_)

0.587292274183266


In [73]:
coeff_df = pd.DataFrame(lm.coef_,shot_df.drop('SHOT_RESULT',axis=1).columns,columns=['Coefficient'])
coeff_df.sort_values(by=['Coefficient'])

,Coefficient
TOUCH_TIME,-0.015481
SHOT_DIST,-0.014570
DRIBBLES,0.006497
CLOSE_DEF_DIST,0.022176


In [74]:
predictions = lm.predict(X_test)

In [81]:
from sklearn import metrics
from sklearn.metrics import r2_score


In [83]:
print('R^2:', r2_score(y_test, predictions))


R^2: -0.05131141079815049
